# Many2one, one2Many， Many2Many的使用


## Many2one(other_model, ondelete=)
- 参数：
    - comodel_name(str)
    - domain
    - context(dict)
    - ondelete(str):'set null', 'restrict', 'cascade'
    - auto_join(bool)
    - delegate(bool)将其设置为True，使目标模型的字段可以从当前模型访问(对应于_inherits）

## One2many(other_model,related_field)
- 参数
    - comodel_name:关联的模型名称
    - inverse_name:管理的模型名称中那个关联的字段名
    - domain
    - context
    - auto_join
    - limit
- 虚拟的关联，即该字段不会在数据库中创建，访问它将返回一个记录集（有可能是空的)。
- 因为他是一个虚拟的关联，所以必须有一个Many2one字段存在于other_model，其名称也必须是related_field。即related_field字段是other_model中必须存在的一个Many2one字段


## Many2many(other_model)
- 参数
    - comodel_name
    - realtion(str）
    - column1
    - column2
    - domain
    - context：上下文(dict)——一个可选上下文，在客户端处理该字段时使用
    - limit (int) -读取时使用的限制
- 双向多对多关联

## self
- self是一个记录的有序集合，它支持标准的Python集合操作，如len(self)和iter(self)，加上额外的集合操作recs1 + recs2。迭代过程逐个提供self记录，其中每个记录本身是大小为1的集合。你可以通过点记号来访问/分配单个记录上的字段


## 默认值：
- fields.xx(default=lambda self: self.env.user)
- self.env对象给出了访问请求参数和其他有用的信息
- self.env.cr或者self._cr是数据库游标对象，通常用于查询数据库
- self.env.uid或者self._uid是当前用户的数据库id
- self.env.user是当前用户记录
- self.env.ref(xml_id)返回xmlid对应的记录
- self.env[model_name]:返回给定模型的实例

## 计算字段
- 在模型的计算型字段，默认是不写入数据库的，不过可以通过store属性来判断，如果store=True，则表示要把该计算字段保存到数据库
- 可以为实时计算字段设置search参数来使其可搜索，参数的值须是一个返回domain表达式的函数
        upper_name = field.Char(compute='_compute_upper', search='_search_upper')

        def _search_upper(self, operator, value):
            if operator == 'like':
                operator = 'ilike'
            return [('name', operator, value)]
- 可以和inverse属性同时使用，不加inverse属性的话，前端界面的显示效果只是只读样式，不能编辑修改；而加上inverse属性时候，相应的关联字段可以修改，但是，修改后，实际上修改的是原表；即会将原表的数据级联更改。

## 关联字段（related fields)
- 可以看成是计算字段的一种特殊情况
- 举例；nickname = fields.Char(related='res.partner.partner_id.name',store=True)
- 关联字段可以设置一个depends参数，指定涉及到某个字段时才触发：
    - nickname = fields.Char(
            related='partner_id.name', store=True,
            depends=['partner_id'])
- 如果没有重新定义，它会从源字段复制一些属性（仅当这些字段是required的时候），所有与语义无关的属性都是从源字段复制的
- 以下两种情况是不允许使用关联字段的：
        m2o_id = fields.Many2one()
        m2m_ids = fields.Many2many()
        o2m_ids = fields.One2many()

        # Supported
        d_ids = fields.Many2many(related="m2o_id.m2m_ids")
        e_ids = fields.One2many(related="m2o_id.o2m_ids")

        # Won't work: use a custom Many2many computed field instead
        f_ids = fields.Many2many(related="m2m_ids.m2m_ids")
        g_ids = fields.One2many(related="o2m_ids.o2m_ids")
        
## 保留字段
- fields.name(Char):作为_rec_name默认值 , 用于在具有代表性“命名”是必须的上下文中显示记录
- fields.active(Boolean):切换全局显示的记录，如果active 被设置为False ，记录在大多的查询和列表中不可见
- fields.state(Section):对象的生命周期阶段，由fields上的states属性使用


# 时间字段
- 方法：
    - Date和Datetime类型共有方法
        - Date方法
            - static_today(timestamp):返回当前用户时区的日期，默认是今天，可选timestamp为datetime类型
            - add(), 时间的加减
            - end_of(granularity), start_of(), granularity can be year, quarter, month, week, day or hour.返回最开始的或最后的时间点
            - subtract()
            - to_date():将字符串或datetime转换为date类型
            - to_string():
            - today():不算入时区
        - Datetime方法
            - context_timestamp(timestamp):返回转换为客户端时区的给定时间戳。
            - now()
            - to_datetime()
            - today()
            - to_string()